In [90]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

In [114]:
def load_data(filename, name, start, stop):
    df = pd.read_csv(filename, parse_dates= [["date", "time"]]).set_index("date_time", drop=True)
    df = df.resample("10T").mean().interpolate(method="linear")
    df = df[(df.index >= start) & (df.index < stop)]
    df = df.rename(columns={"value":name})
    print("Filename:", filename)
    print(df.shape, df.iloc[0], df.iloc[-1])
    print()
    return df

def uncumulate(df):
    df = df.diff()
    return df.fillna(df.iloc[1,0])

In [115]:
start = datetime(2021, 3, 9)
stop = datetime(2021, 3, 16)

filenames = ["./air_temp.csv", "./elspec_active_import.csv", "./GHI.csv", "./P1P2_active.csv", "./P2_active.csv", "./P2_load_import.csv", "./PV_export.csv", "./SOC.csv", "./wind_speed.csv"]
column_names = ["airTemp","elspec", "GHI", "L", "L2_active", "L2_import", "PV", "SOC","Wind_speed"]
air_temp, elspec, GHI, P1P2, P2, P2_import, PV, SOC, wind_speed = [load_data(filenames[i], column_names[i], start, stop) for i in range(len(filenames))]

P2_import = uncumulate(P2_import)
PV = uncumulate(PV)

Filename: ./air_temp.csv
(1008, 1) airTemp   -5.5
Name: 2021-03-09 00:00:00, dtype: float64 airTemp    1.5
Name: 2021-03-15 23:50:00, dtype: float64

Filename: ./elspec_active_import.csv
(1008, 1) elspec    181110.393519
Name: 2021-03-09 00:00:00, dtype: float64 elspec    216840.131944
Name: 2021-03-15 23:50:00, dtype: float64

Filename: ./GHI.csv
(1008, 1) GHI    2.967105
Name: 2021-03-09 00:00:00, dtype: float64 GHI    4.770833
Name: 2021-03-15 23:50:00, dtype: float64

Filename: ./P1P2_active.csv
(1008, 1) L    138.505572
Name: 2021-03-09 00:00:00, dtype: float64 L    117.804629
Name: 2021-03-15 23:50:00, dtype: float64

Filename: ./P2_active.csv
(1008, 1) L2_active    13.994041
Name: 2021-03-09 00:00:00, dtype: float64 L2_active    11.600739
Name: 2021-03-15 23:50:00, dtype: float64

Filename: ./P2_load_import.csv
(1008, 1) L2_import    914213.546196
Name: 2021-03-09 00:00:00, dtype: float64 L2_import    916726.440789
Name: 2021-03-15 23:50:00, dtype: float64

Filename: ./PV_export

In [120]:

df = pd.concat([air_temp, elspec, GHI, P1P2, P2, P2_import, PV, SOC, wind_speed], axis =1)

df.to_csv("../../src/data/09.03_cleaned.csv", index=False)